In [1]:
#Based on the optimized1 process. We eliminated several independent variables.
#As much of the data from Thailand lacks a linear relationship with the Crop Yield, we decided to drop the Thailand Data from the combined model.

#In addition we dropped the following columns of data:
#Energy
#Fert
#Air Deposition of Nitrogen
#Pesticide

###adj r^2: 0.9004562805076817 (before) => 0.835540504866495 (after)

In [2]:
%run MLR_Functions.ipynb

In [3]:
import pandas as pd

In [4]:
df_rice_y = pd.read_csv("/Users/yxlsa/Desktop/SUTD/DTPIII/Cleaned Data/df_crop_yield.csv")
#df_fert_use = pd.read_csv("/Users/yxlsa/Desktop/SUTD/DTPIII/Cleaned Data/df_fertilizer_usage.csv")
df_cropland = pd.read_csv("/Users/yxlsa/Desktop/SUTD/DTPIII/Cleaned Data/df_cropland.csv")
#df_energy = pd.read_csv("/Users/yxlsa/Desktop/SUTD/DTPIII/Cleaned Data/df_energy_usage.csv")
df_pop = pd.read_csv("/Users/yxlsa/Desktop/SUTD/DTPIII/Cleaned Data/df_urban_rural_pop.csv")
df_water = pd.read_csv("/Users/yxlsa/Desktop/SUTD/DTPIII/Cleaned Data/df_water_usage.csv")
#df_pesticide = pd.read_csv("/Users/yxlsa/Desktop/SUTD/DTPIII/Cleaned Data/df_pesticide_usage.csv")
df_pesticide_imports = pd.read_csv("/Users/yxlsa/Desktop/SUTD/DTPIII/Cleaned Data/df_pesticide_imports.csv")
#df_air_dep = pd.read_csv("/Users/yxlsa/Desktop/SUTD/DTPIII/Cleaned Data/df_air_deposition_nitrogen.csv")

In [5]:
#combine column for yield
df_rice = pd.DataFrame({
    'Yield': pd.concat([df_rice_y['India Rice Yield'], df_rice_y['China Rice Yield'], df_rice_y['Vietnam Rice Yield']], ignore_index= False)
})

df_rice

,Yield
0,2.6125
1,2.6271
2,2.6092
3,2.8303
4,2.8645
...,...
25,5.7597
26,5.5738
27,5.5476
28,5.8180


In [6]:
#rural and urban
df_rural = pd.DataFrame({
    'Rural Pop': pd.concat([df_pop['india rural/total'], df_pop['china rural/total'], df_pop['vietnam rural/total']], ignore_index= False)
})

df_rural

,Rural Pop
0,0.744253
1,0.741475
2,0.738890
3,0.736265
4,0.733634
...,...
25,0.671822
26,0.665044
27,0.658261
28,0.651453


In [7]:
df_urban = pd.DataFrame({
    'Urban Pop': pd.concat([df_pop['india urban/total'], df_pop['china urban/total'], df_pop['vietnam urban/total']], ignore_index= False)
})

df_urban

,Urban Pop
0,0.255381
1,0.257528
2,0.259399
3,0.261260
4,0.263131
...,...
25,0.343149
26,0.350446
27,0.357773
28,0.365160


In [8]:
#water combined into one column
df_gimme_da_water = pd.DataFrame({
    'Water': pd.concat([df_water['wf_tot_m3_t_india'], df_water['wf_tot_m3_t_china'], df_water['wf_tot_m3_t_vietnam']], ignore_index= False)
})

df_gimme_da_water

,Water
0,1398.451456
1,1412.468478
2,1369.645800
3,1290.836077
4,1357.477217
...,...
25,731.201526
26,750.798534
27,670.644750
28,676.491184


In [9]:
#combine column for Pesticide Imports
df_get_pesticide_imports = pd.DataFrame({
    'Pesticide Import': pd.concat([df_pesticide_imports['India'], df_pesticide_imports['China'], df_pesticide_imports['Vietnam']], ignore_index= False)
})

df_get_pesticide_imports

,Pesticide Import
0,0.014590
1,0.010638
2,0.009107
3,0.010173
4,0.025008
...,...
25,13.702846
26,13.498934
27,9.662465
28,19.032825


In [10]:
combined_df = pd.concat([df_rice,df_rural,df_urban,df_gimme_da_water,df_get_pesticide_imports], axis = 1)
combined_df.reset_index(drop=True, inplace= True)
display(combined_df)
#print(type(combined_df))
#print(combined_df.shape)
#print(combined_df.loc[0,:])

,Yield,Rural Pop,Urban Pop,Water,Pesticide Import
0,2.6125,0.744253,0.255381,1398.451456,0.014590
1,2.6271,0.741475,0.257528,1412.468478,0.010638
2,2.6092,0.738890,0.259399,1369.645800,0.009107
3,2.8303,0.736265,0.261260,1290.836077,0.010173
4,2.8645,0.733634,0.263131,1357.477217,0.025008
...,...,...,...,...,...
85,5.7597,0.671822,0.343149,731.201526,13.702846
86,5.5738,0.665044,0.350446,750.798534,13.498934
87,5.5476,0.658261,0.357773,670.644750,9.662465
88,5.8180,0.651453,0.365160,676.491184,19.032825


In [11]:
# Extract the features and the target
df_features, df_target = get_features_targets(combined_df,["Rural Pop", "Urban Pop", "Water", "Pesticide Import"],["Yield"])

# Split the data set into training and test
df_features_train, df_features_test, df_target_train, df_target_test = split_data(df_features, df_target, random_state = 100, test_size = 0.3)

#print(df_features_train)
#print(df_features_test)

# Normalize the features using z normalization
array_features_train_z, means, stds = normalize_z(df_features_train)

# Change the features and the target to numpy array using the prepare functions

X: np.ndarray = prepare_feature(array_features_train_z)
target: np.ndarray = df_target_train.to_numpy()

iterations: int = 1500
alpha: float = 0.01
beta: np.ndarray = np.zeros((5,1))

# Call the gradient_descent function
beta, J_storage = gradient_descent_linreg(X, target, beta, alpha,iterations)

# call the predict() method
pred: np.ndarray = predict_linreg(df_features_test,beta, means,stds)

In [12]:
# change target test set to a numpy array
target: np.ndarray = df_target_test.to_numpy()

# Calculate r2 score by calling a function
r2: float = r2_score(target, pred)
    
# Calculate adj r2
#Set the n and k
n = 90
k = 4
adj_r2: float = adj_r_squared(n,k,target,pred)

# Calculate the mse
mse: float = np.mean((target-pred)**2)
    
# Calculate the mae
mae: float = np.mean(np.absolute(target-pred))
    
print(f"r2: {r2}")
print(f"adj_r_squared: {adj_r2}")
print(f"mse: {mse}")
print(f"mae: {mae}")

r2: 0.8429319428500233
adj_r_squared: 0.835540504866495
mse: 0.28983349607818604
mae: 0.4424805837583485


In [13]:
print(pred)
print(df_target_test)
print(df_features_test)

[[7.00485469]
 [2.75531198]
 [4.70529926]
 [5.07358985]
 [5.91416962]
 [7.00255838]
 [5.52028477]
 [5.86320428]
 [4.32245899]
 [4.99981975]
 [2.9101442 ]
 [5.09532415]
 [2.92779409]
 [3.43893161]
 [4.74319084]
 [4.902534  ]
 [5.81106796]
 [4.68881611]
 [5.41150186]
 [5.35013386]
 [4.4278397 ]
 [4.61004865]
 [2.69221964]
 [2.97608138]
 [5.785626  ]
 [2.74730648]
 [4.28143107]]
     Yield
57  6.9169
61  3.1133
26  3.7401
29  4.0837
43  6.0607
56  6.8657
35  6.0248
44  6.3106
22  3.6909
75  4.8891
5   2.6972
32  5.8031
11  3.1158
65  3.6898
28  3.9568
73  4.6387
36  6.2122
69  4.1018
78  5.2336
77  4.9869
20  3.3587
23  3.6243
6   2.8226
3   2.8303
40  6.2716
12  2.6163
25  3.6002
    Rural Pop  Urban Pop        Water  Pesticide Import
57   0.414563   0.584806   725.787876          0.641679
61   0.808945   0.210249  1290.906476          0.828139
26   0.660956   0.328238   995.647590          0.543529
29   0.648472   0.341135   931.902423          0.640366
43   0.601400   0.411548   820.11

In [15]:
###61  3.1133
###61   0.808945   0.210249  1290.906476          0.828139

country_yield = 3.1133
rural_pop = 0.808945
urban_pop = 0.210249
water = 1290.906476
pesticide_imports = 0.828139

def normalise(feature_value:int, mean, std):
    return (feature_value-mean)/std
    

def get_crop_yield(rural_pop, urban_pop, water, pesticide_imports, beta):
    rural_pop_n = normalise(rural_pop ,means[0],stds[0]) 
    urban_pop_n = normalise(urban_pop ,means[1],stds[1])
    water_n = normalise(water ,means[2],stds[2])
    pesticide_imports_n = normalise(pesticide_imports, means[3],stds[3])
    y = beta[0] + beta[1]*rural_pop_n + beta[2]*urban_pop_n + beta[3]*water_n + beta[4]*pesticide_imports_n
    return y

print(get_crop_yield(rural_pop, urban_pop, water, pesticide_imports, beta))

[2.7553107]
